In [1]:
import requests
import pandas as pd
import os
from tqdm.auto import tqdm
import json
import math
import sys

sys.path.append('..')
from src.textprocessing.preprocess import normalize_text, remove_punctuation
pd.set_option("display.max_colwidth", None)

/Users/matiasgonzalez/miniforge3/envs/llmsalary/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matiasgonzalez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Set up chatgpt model
# My token
os.environ['CHAT_GPT_API_KEY'] = ""

from src.api import process_request

NGRAMS = 5
OVERLAP = 0.3

Params NGRAMS: 3 | OVERLAP: 0.3


In [3]:
def create_job_rerun_id(row):
    return "rerun_"+row.job_id

In [4]:
EXPORT_NAME = "latam_004"
SOURCE_BASE_DIR = "./datasets"
VERSION_MODEL = "0.8.1"
EXTRACT_FOLDER = "extraction_004" #"extraction_001_002_003"
FILE_COUNT = len([filename for filename in os.listdir(SOURCE_BASE_DIR+os.sep+EXTRACT_FOLDER)])-1
HEADERS = ["reduce_description", f"salary_min_gpt_{VERSION_MODEL}", f"salary_max_gpt_{VERSION_MODEL}", f"currency_gpt_{VERSION_MODEL}", f"time_lapse_gpt_{VERSION_MODEL}"]

#Concat
discarted_rows_list = []

In [5]:
# Read processed_ids so far
processed_df = pd.concat([pd.read_csv(path, engine="python") for path in [SOURCE_BASE_DIR+os.sep+EXTRACT_FOLDER+os.sep+f"raw_extract_n_{n+1}.csv" for n in range(FILE_COUNT)]])
processed_df = processed_df.dropna(subset="job_id")
processed_df = processed_df.dropna(subset="description")
processed_df["description_normalized"] = processed_df.description.apply(normalize_text)
processed_df["description_normalized"] = processed_df.description.apply(remove_punctuation)
processed_df = processed_df.drop_duplicates(subset=["description_normalized"])
print(f"Preprocessed jobs: ", processed_df.shape[0])
OG_SIZE = processed_df.shape[0]

Preprocessed jobs:  57084


In [6]:
processed_df[processed_df["salary_min_gpt_0.8.1"]!=0].shape

(12653, 30)

In [7]:
INIT_PERCENTAJE_JOBS_WITH_SALARIES = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]!=0].shape[0]/processed_df.shape[0]

In [8]:
processed_df.columns

Index(['Unnamed: 0', 'job_id', 'reduce_description', 'salary_min_gpt_0.8.1',
       'salary_max_gpt_0.8.1', 'currency_gpt_0.8.1', 'time_lapse_gpt_0.8.1',
       'source_0.8.1', 'token_count_sent_chat_gpt_0.8.1',
       'token_count_original_description',
       'token_rate_sent_original_description', 'gpt_cost_0.8.1', 'title',
       'company_name', 'location', 'via', 'description', 'job_highlights',
       'related_links', 'thumbnail', 'extensions', 'posted_at',
       'schedule_type', 'work_from_home', 'country_search', 'salary',
       'commute_time', 'date_posted', 'tech_skills', 'description_normalized'],
      dtype='object')

In [9]:
processed_df["currency_gpt_0.8.1"].unique()

array(['USD', ' ', nan, 'anywhere', '-', 'ARS', 'EUR', 'days', 'BRL',
       'COP', 'simultaneous projects', '%', 'hour', 'job openings', 'UTC',
       'calendar days', 'hours', 'entravision', 'month', 'MXN',
       'percentage', 'RON', 'freelance', 'about cognizant', 'CLP', 'PLN',
       'Madrid Time', 'SAR', 'SGD', 'ARG', 'Not Required', 'VAL', 'BOB',
       'DOP', 'local currency', 'employees', 'year', 'number', '{}',
       'lookingfor', 'GBP', 'oowlish paradise', 'CAD', 'ANY', 'ZAR',
       'minutes', 'flex', 'Global', 'professionals', 'customers', 'CAN',
       'English', 'employee', 'years', 'full-time', 'experience', 'INR',
       'shakers', 'weeks', 'hits', 'mile', 'Toronto Canada', 'vacation',
       'device', 'people', 'moments', 'views', 'any', 'PTO', 'Canada',
       'TOR', 'ISO 22716', 'AST', 'contractor', 'BWP', 'ET', 'Anywhere',
       'FLC', 'person', 'engineers', 'clients', 'unknown', 'technologies',
       'VND', 'COL', 'permanent', 'BHD', 'CRC', 'developers', 'month

# El modelo de chat gpt entiende el currency como una unidad de cantidad de algo

In [10]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="UTC"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
1261,"est (utc -5) mst (utc uzt (utc +5) irdt (utc mst (utc -7) art (utc wib (utc +7) mmt (utc art (utc -3) utc -4 -4:30 utc -3 utc -2 msk (utc +3) jst (utc vuejs about 3 years ago, loop with 3 x team -3) utc -4 utc -4:30 gst (utc +4) sporty's sites -4 utc -4:30 utc -3 irdt (utc +4:30) gst (utc -4:30 utc -3 utc -2 msk (utc +3) jst (utc -3 utc -2 sbt (utc eet (utc +2) msk (utc minimum of 2+ years within annual leave 2 x annual phuket in q2 2023 + sbt (utc +11) gmt (utc gmt (utc +0) cet (utc cet (utc +1) eet (utc 2023 + 1 more tbc!) eet (utc +2) msk (utc annual leave 2 x annual -4:30 utc -3 utc -2 msk (utc +3) jst (utc jst (utc +9) cst (utc cst (utc +8) wib (utc drone ci, k8s responsibilities develop mst (utc -7) art (utc wib (utc +7) mmt (utc mmt (utc +6:30) bst (utc bst (utc +6) npt (utc with git, es6, webpack, less npt (utc +5:45) ist (utc ist (utc +5:30) uzt (utc est (utc -5) mst (utc uzt (utc +5) irdt (utc -4 utc -4:30 utc -3 irdt (utc +4:30) gst (utc -3) utc -4 utc -4:30 -4:30 utc -3 utc -2 msk (utc +3) jst (utc cst (utc +8) wib (utc eet (utc +2) msk (utc annual leave 2 x annual bst (utc +6) npt (utc referral bonuses 28 days paid eet (utc +2) msk (utc annual leave 2 x annual dubai in 2022 / phuket eet (utc +2) msk (utc annual leave 2 x annual in q2 2023 + 1 cet (utc +1) eet (utc we score 100% on the check remote 90 min video -4:30 utc -3 utc -2 msk (utc +3) jst (utc team members (30 mins each) via zinc 24-72 hour feedback anywhere",-430.0,-300.0,UTC,hour


In [11]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="REQ"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1


In [12]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="Percentage"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1


In [13]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="Salarios mínimos"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1


In [14]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="PLN"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
2686,"got a 75% flex score. top performers. 100% of our salary: pln 17.000 - 24.000 17.000 - 24.000 + vat + vat (b2b) required skills: english (cefr b2+) and polish fully remote b2b/uzl: + paid at least 2 projects involving have minimum b2 english - at least 4 years of and polish (c1) nice to polish level c1 minimum know and benefits: +6 000 pln benefits: +6 000 pln / budget, + 1500 pln / in our phase0 (discover - - used s3, cognito, sns, anywhere",17000.0,24000.0,PLN,month
1803,requirements minimum 2 years of pln/hour (net) b2b contract type we offer 60-110 pln/hour (net) calgary ab canada,60.0,110.0,PLN,hour
2349,"rabbit mq b2b remote possible: api and e2e tests in experience with git2+ years experience with robot framework2+ years experience remote possible: 60% your tasks: experience with git2+ years experience looking for?: 3+ years of web and api3+ years experience with database technologies3+ years experience experience with git2+ years experience experience with git2+ years experience english, polishcontract: 145 pln/h + toronto canada",145.0,145.0,PLN,hour
192,"makes the web3 world a analytics with ga4 data; data at least 5 years of between pln 19 000 - pln 19 000 - 30 - 30 000 net on 000 - 30 000 net growing company, 26+ fully paid fit in? 90%+ of ramps anywhere",19000.0,30000.0,PLN,year
198,remote job 8.0k-10.1k (b2b) pln job 8.0k-10.1k (b2b) pln / anywhere,8000.0,10100.0,PLN,month


In [15]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="preferred"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1


In [16]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"]=="Unknown"].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1


# valid currencies

Con lo cual filtrar primero por currency que representa dinero deberia ser el primer filtro

In [17]:
allowed_currencies = ['MXN', 'CLP', 'USD', 'COP', 'MVC', 'DOP', 'EUR', 'GTQ', 'CRC', 'KRW', 'SVC', 'HNL', 'UYU', 'PEN',
                     'BRL', 'ARS', 'NTQ', 'HRS', 'QAR', 'GBP', 'PHP', 'CAD', 'JMD', 'BMD', 'MMK',
                     'ZAR', 'NIO', 'PAB', 'PYG', 'MYR', 'PLN', 'AED', 'INR', 'CST', 'CHF', 'BOB']

In [18]:
processed_df = processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"].isin(allowed_currencies)]

print(processed_df.shape)

(14158, 30)


In [19]:
def create_discarted_df(discarted_rows_list):
    return pd.concat([pd.DataFrame(df) for df in discarted_rows_list])

def get_discarted_sample(discarted_df, HEADERS):
    return discarted_df[discarted_df[f"salary_min_gpt_{VERSION_MODEL}"]!=0].sample(n=10)[HEADERS]

# Quedamos con aprox un 23% de avisos que decimos que tiene salarios

In [20]:
round(processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]!=0].shape[0]/OG_SIZE, 2)

0.21

# Random sample

In [21]:
processed_df.sample(n=10)[["reduce_description", f"salary_min_gpt_{VERSION_MODEL}", f"salary_max_gpt_{VERSION_MODEL}", f"currency_gpt_{VERSION_MODEL}", f"time_lapse_gpt_{VERSION_MODEL}"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
1346,"for more than 1 billion users on reputable institutions. - 3+ years of hands-on selected city is $159000 - $230000 annually. employees: we cover 100% premium coverage for leave plans are: 10 paid holidays per by tenure) and 10 paid sick days per year plus 17 days of paid as well as 12 weeks of paid and lyra. a 401k company match, gym san jose province san jose costa rica",159000.0,230000.0,USD,year
883,"por hora inicial: $15.00 usd all about as a fortune 50 company with more with more than 350,000 team members worldwide, be at least 18 years of age merchandise up to 40 pounds flexible work guest services at 1-800-440-0680 for additional information. west des moines ia",15.0,15.0,USD,hour
1013,"engineering concepts and 5+ years of applied light of the covid-19 pandemic. jpmorgan chase to address a post-covid-19 environment. as a the firm's current covid-19 or other infectious all of our 6,200 locations globally based base pay/salary ny $147,250.00 - $215,000.00 / new york ny",147250.0,215000.0,USD,NaN
3174,"or related field 6+ years of experience this role is $129,300 to $194,000. this atlanta ga",129300.0,194000.0,USD,NaN
2944,"of circa 5 billion and more than 53 countries. by continuum with 57,000 world-class talents, france in 2005, the company more than 300 international climate minimum of 6 years experience for scopes 1 and 2 1 and 2 and scope and scope 3 categories. knowledge and global 500 clients, including on assignment (<10%). salary and mississauga canada",57000.0,57000.0,CAD,year
211,"requires being in-office 5 day a week... outlook), imanage/interwoven desksite 5, and changepro possess programs, including windows 7 and microsoft office minimum of three (3) years of technology this role is $90,000 - $100,000 and partnership contributions to 401(k) accounts may be and more) generous 401(k) plan and firm-paid receive: at least 11 paid holidays per program that accrues 23 days during the san francisco ca",90000.0,100000.0,USD,NaN
2715,"equivalent practical experience. 5 years of experience software products, and 1 year of experience in more than 200 countries and territories full-time position is $157,000-$235,000 + bonus + new york ny",157000.0,235000.0,USD,NaN
2625,"salary/hourly wage: *$80k-$100k based on heroku, aws s3 project management professional experience 4+ years of that's between 11-50 employees. you anywhere",80000.0,100000.0,USD,hour
4555,"truly hands-free. 10,000+ organizations, from more than 10 million payments to over 220 countries and ratings on g2. yet theres challenge. we're ~80 people, highly role is $100,000 - $240,000 $100,000 - $240,000 we're a that's between 51-200 employees. you anywhere",100000.0,240000.0,USD,year
3494,"sharepoint developer, #661 clearance: u.s. citizenship current environment has 1 site collection, with collection, with approximately 63 subsites. o365 sharepoint bachelors degree and 5 years of related ability with sharepoint 2016, sp designer, browser, this position is $72,000 - 107,000, this insurance, and more 401k matching with immediate frederick md",72000.0,107000.0,USD,NaN


# time lapse

El segundo nivel es timelapse que puede remover alguna que otra incosistencia

In [22]:
processed_df[f"time_lapse_gpt_{VERSION_MODEL}"].unique()

array(['year', nan, 'hour', 'week', 'month', 'day', 'position',
       'real-time', 'interview', 'January 2021', '12months', '6 months',
       'certification',
       'android (version 4.1 or higher) smartphone (version 8 or higher)',
       '3+ years', 'trillion', 'round', '5-7 years', '5+ years', 'person',
       'anywhere', 'publication_date', 'series a', 'days', 'minute',
       'months', '36', '3 months', 'percent', '3/4 months', '7 cycles',
       'hackathons', 'double bonus', 'annually', 'wy', 'weeks', 'course',
       'specified period', 'per equivalent experience', 'business days',
       '6-month', 'individual', 'once', 'quarter', 'session',
       '12+ months', '6 month', 'referral_fee', 'annum', 'anuum',
       'bi-weekly', 'job', 'eight months', 'commission', '6+ months',
       'hiram', 'awards', 'September 5th to December 15th, 2023',
       'permanent', '%', '9 month', 'variable', 'foot', 'annual',
       '3 weeks', 'travel', 'shift', 'statutory holidays', 'working day

In [23]:
valid_time_lapses = ['month','year', 'hour', 'annual']

In [24]:
discarted_rows_list.append(processed_df[~processed_df[f"time_lapse_gpt_{VERSION_MODEL}"].isin(valid_time_lapses)])

processed_df = processed_df[processed_df[f"time_lapse_gpt_{VERSION_MODEL}"].isin(valid_time_lapses)]

print(processed_df.shape)

(7162, 30)


In [25]:
# ahora nos quedamos con el 21%

round(processed_df.shape[0]/OG_SIZE, 3)

0.125

In [26]:
processed_df.sample(n=10)[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
3892,"degree and or 4+ years of experience https://myhperewards.com/main/new-hire-enrollment.html. annual salary: $111,900.00 - $252,000.00 hewlett united states",111900.00,252000.00,USD,year
4005,"who we are 2k is headquartered in responsible for developing 2k's portfolio of world-class all of us! 2k publishes titles in such as nba 2k, 2k pga, battleborn, midnight suns, wwe 2k, and xcom. at to work at 2k, we encourage you engineering group for 2k publishing's global services. connective tissue across 2k's studios! in short, projects across the 2k ecosystem, occasionally supporting with programmers across 2k to build tools, please note that 2k games and its emailing, only use 2k.com accounts. #li-hj1 #li-onsite concepts, firaxis, hangar 13, catdaddy, cloud chamber, global services. since 1995, we've developed deep skills in c/c++ 5+ years of experience systems such as 3d math, audio processing, to be between $87,400 and $157,300 per novato ca",87400.00,157300.00,USD,year
3773,"testing experience: 3+ years in a stamp 4 visa or role. salary/remuneration 40,000-50,000, for more anywhere",40000.00,50000.00,EUR,month
2882,through friday from 8:00 to 5:00 pm. pay ranges from $18.00 to $25.00 per o'fallon mo,18.00,25.00,USD,hour
3913,technician. pay rate: $44hr... required qualifications: we commuter benefits and 401k plan. our program california,44.00,44.00,USD,hour
1986,this is an 8-month contract opportunity with requirements typically requires 7 years of relevant experience. job requisition #37361 a reasonable estimate this role is $42.50 - $47.50 per apc company staffing 501 1000 employees apc bloomington il,42.50,47.50,USD,hour
3041,"mn area... pay: $66.55/hr c2c job description: must be your w2 employee :yes exclusive certifications like win 95/98/nt/unix, java script, vb st paul mn",66.55,66.55,USD,hour
3222,position is cad $33.66 - $48.38 / knox mobile enrollment 2 to 4 years science with two (2) to four (4) surrey bc canada,33.66,48.38,CAD,hour
1708,& traits 2+ years of experience: copywriting: 2 years (required) part-time hours: 20 per week week pay: $20.00-$25.00 per hour anywhere,20.00,25.00,USD,hour
4178,"engineer - aws/k8/terraform | singularis engineer - aws/k8/terraform infraestrutura remoto refeicao de r$477,97 e vale alimentacao de r$150,00, totalizando r$627,97 caju de r$150,00 mensais; programa r$150,00, totalizando r$627,97 mensais no horaria semanal 44 horas regime - recrutei 2023 brazil",627.97,627.97,BRL,month


# Por rango general

In [27]:
processed_df = processed_df.astype({f"salary_min_gpt_{VERSION_MODEL}": "float64", f"salary_max_gpt_{VERSION_MODEL}":"float64"})

In [28]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>0]

print(processed_df.shape)

(6836, 30)


In [29]:
#40 or 45 hours week
for week_hours in [25, 40, 45]:
    week_hours = processed_df[(processed_df["reduce_description"].str.contains(f"{week_hours} hours"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==week_hours)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==week_hours))]
    processed_df = processed_df[~processed_df.job_id.isin(week_hours.job_id)]

#years
for age in range(50):
    years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} years"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==age))]
    processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

for age in range(5000):
    years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} fastest"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==age))]
    processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

# Numeros que representan ganancias u otras cosas

In [30]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>=1000000000].head()[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
999,"transform the $125b information security looking for 5 years of dental, vision, 401k matching, wellness, sick days, 13 holidays collaborating retained for 6 months (12 6 months (12 months with",1.250000e+11,1.250000e+11,USD,year
3393,"more than $1 billion. and launching in 2003, weve been in over 160 countries. and in over 120 countries, with you have: 4+ years of network, routing, l3/l7 protocols, etc.",1.000000e+09,1.000000e+09,USD,year
4487,"each year, 2 billion people our recent $2bn valuation, were you: minimum 2 years working at over $2bn. quarterly travel progressive fortune 100 and 500 100 and 500 companies, as as over +1400 other established equity shares 401(k) discretionary pto brazil",2.000000e+09,2.000000e+09,USD,year
446,more than $1bn during their role requires 4-5+ hours overlap at least 5+ years of at least 5 years of anywhere,1.000000e+09,1.000000e+09,USD,hour
1795,"areeach year, 2 billion people our recent $2bn valuation, were you: minimum 2 years working at over $2bn. quarterly travel progressive fortune 100 and 500 100 and 500 companies, as as over +1400 other established insurance equity shares401(k)discretionary pto planparental brazil",2.000000e+09,2.000000e+09,USD,year


In [31]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>=1000000000].shape[0]

10

In [32]:
# Total son al rededor de 19 de 62894

processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>=1000000000].shape[0]

# representan el
print(round(processed_df.shape[0]/OG_SIZE, 4))

0.1179


In [33]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<1000000000]

print(processed_df.shape)

(6723, 30)


In [34]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>=50000000].sample(n=10)[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
2299,mas de 50 millones de,50000000.0,5.000000e+07,COP,month
1315,more than $78 mn during sao paulo state sao paulo brazil,78000000.0,7.800000e+07,BRL,month
4756,"are investing $100 million annually of experience (3+ years for masters or 6+ years for preferred qualifications 5+ years practical have over 60,000 tbs worth working environment (2-3 days a total of 8 weeks in a year, 4 of those the world. #li-ed1 do you for almost 160 years. our talents of 25,000 employees across more than 75 countries, where sao paulo state sao paulo brazil",100000000.0,1.000000e+08,USD,year
4212,more than $152mn during their position requires 6+ hours of at least 3+ years of business analysis 2+ years of anywhere,152000000.0,1.520000e+08,USD,year
3070,"back to 1907 with a for over 60 years - solutions in 60 countries, employing are in 30 countries worldwide with between 15 - 20 15 - 20 years' experience at least 10 years increasing is required ($100mn-$1 bn) mining minimum of 5 years mining employing around 35,000 people. www.woodplc.com saskatchewan canada",100000000.0,1.000000e+09,USD,year
3258,mas de 50 millones de,50000000.0,5.000000e+07,COP,month
399,"inception in september 2016, we have grown start-up to a $100m/year business. alpha omegas accolades including inc. 5000 and washington technologys washington technologys fast 50 awards for five the year over $50 million dollars award, maryland technology council's 2022 government contract of proficiency with: oracle 11g/12c databases pl/sql packages, jquery, ajax, xml, html5, css, soap based screen required skill/experience: 2-3 years hands-on experience women, people over 40, and differently-abled folks to furnish information. 41 cfr 60-1.35(c anywhere",100000000.0,1.000000e+08,USD,year
3097,"**nosotros** desde 2013, edunext ha tenido heredadas. **requisitos**: - 2 anos de experiencia python - nivel b2 en ingles - (como javascript y html5) **algunos skills a remoto - remuneracion $5.500.000 mensuales o 1400 indefinido, freelance salario: $550.000.000 al mes pregunta(s) bogota bogota colombia",550000000.0,5.500000e+08,COP,month
4445,"y manejo en html5 y css conocimiento a viernes de 08:30 a 18:30 hrs. hace mas de 10 anos con una nuestros mas de 800 profesionales altamente cualificados lo requieran (ley 20.422). unete a nuestro completo, indefinidio salario: $150.000.000 - $190.000.000 al consideraciones ante el covid-19: mascarilla obligatoria pregunta(s) limite para postularse: 19-02-2022 maule chile",150000000.0,1.900000e+08,CLP,month
3054,more than $250mn in funding at least 6+ years of software developer 6+ years of anywhere,250000000.0,2.500000e+08,USD,year


In [35]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>=50000000].shape[0]

print(processed_df.shape[0])

6723


In [36]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<50000000]

print(processed_df.shape)

(6681, 30)


In [37]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<20000000]

print(processed_df.shape)

(6668, 30)


In [38]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<10000000]

print(processed_df.shape)

(6637, 30)


In [39]:
# nos deja con un 21% del total de avisos con salarios

print(round(processed_df.shape[0]/OG_SIZE, 3))

0.116


# Sample

In [40]:
processed_df.sample(n=10)[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
2120,"term: 3 months could be works online, part-time 3 hours per day on performance pay: $800 per month works bid. max bid $800, do not bid database.. more than 4 years experience with anywhere",800.0,800.0,USD,month
3067,"pay range: $90-$100/hr intelliswift software objectives. top 3 must-have hard organizational skills 3 influencing & hard skills: 1 project management have skills: 1 nice-to-have skills: project management 2 organizational skills proficiency (excel) 2 successful background: & communication 4. sme source to pay 5, bachelors degree, bachelors degree, 7-10 years experience have skills: 1 nice-to-have skills: anywhere",90.0,100.0,USD,hour
430,at least 2 years of at least 2 hours of range is $600 - $800 $600 - $800 based on shift of 9am-3pm est (us). brazil,600.0,800.0,USD,hour
4949,"date: october 3, 2022 salary october 3, 2022 salary range: date: december 2022 this position october 16, 2022. while we salary range: $67,221- $116,532 (as range: $67,221- $116,532 (as per of the 2slgbtq community. if received by 11:59 p.m. on on october 16, 2022. while barrie canada",67221.0,116532.0,CAD,month
550,"and got a 82.75% flex score... dealership: location: remote compensation: $50,000 responsibilities set up paid life insurance 401(k) retirement plan employee graduate or equivalent, 18 years or older anywhere",50000.0,50000.0,USD,year
4346,"which touch nearly 1,800 lives annually include education. mckinley has 5 locations throughout southern were offering is $67,000 - $77,000 annually defined contribution of $550/month life insurance flexible time paid holidays 403(b) retirement plan with match up to 3% employee assistance program ability to lift 10 pounds for filing knowledge of title 22 gh standards, dmh california mo",67000.0,77000.0,USD,year
279,"customers in over 190 countries, including several space. estimated salary: $20 to $28 per san francisco ca",20.0,28.0,USD,hour
3371,"minimum of 4 years experience at unit 403, 2333 dundas unit 403, 2333 dundas st. toronto, on m6r 3a6; this on m6r 3a6; this will compensated at $115,000 per year, minimum of 40 work hours that's between 11-50 employees. you ontario canada",115000.0,115000.0,CAD,year
4337,"to mid-september 2023 (with potential december), with 8-hour workdays. requirements than may 2024 familiarity with rate of $18 fee drinks calgary ab canada",18.0,18.0,CAD,hour
3350,"technology posting date: 07/28/2023... closing date: 08/10/2023 dependent upon position) 10 -25 days of vacation time annually (10 days for first on election years 10 weeks of paid resolve system problems. 10% keeps abreast of date: 08/10/2023 salary: $6,543- $9,936/monthly job type: number of vacancies: 1 plan/bu: rc063 this above. minimum qualifications 1. requires knowledge, skill, field. specialized skills 1. requires three (3) 2. requires one (1) year of professional conditions of employment 1. requires ability to electronically through the illinois.jobs2web.com website. state of non-state employees: on illinois.jobs2web.com click application procedures posting identification number 28987 **transcripts are required. of state employment) 12 days of paid year to year 3 paid personal business and web services. 3. three (3) years days per year 13-14 paid holidays per link: job responsibilities 35% under administrative direction, issues, as necessary 30% conducts and coordinates (job responsibilities continued) 20% provides technical guidance when held off-site. 5% performs other duties or research nature. 5. ability to gain completion of four (4) years of college plus, and pl/sql. 4. ability to analyze clients, and others. 6. developed verbal and clarity and precision. 7. demonstrated organizational skills location: 801 s 7th st springfield, il etc.). work hours: 8:30 am - 5:00 services. with over 1,500 employees, dolt delivers springfield il",6543.0,9936.0,USD,month


In [41]:
processed_df.sample(n=10)[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
112,"(ui) using html4/html5, javascript, jquery, tested qualifications: 7+ years hands industry, including 100 remote work remote work 100 payment usd",100.0,100.0,USD,month,mendoza capital department mendoza province argentina
2211,"auto-id products. our 30,000+ customers include startups startups and fortune 100 companies alike, and products to over 100 countries. since our our founding in 2008, we've been on business ethics codes 0 - 90 day type: part-time pay: $17.00 per hour schedule: birmingham",17.0,17.0,USD,hour,birmingham
4770,"and got a 80% flex score... sellermetrics in the top 100 for their niche, the team, requiring 100% commitment, this is user count scales 3-5 times. improving the upload your cv 3 - short interview 9 am and 5 pm. in this test, if short-listed 5 - attending video working hours from 9 am to 6 for at least 30 days to ensure this role is 4.5k to 5.5k, providing process to include: 1 - filling in a technical recruiter 4 - coding test, anywhere",4500.0,5500.0,USD,month,anywhere
4111,"to be between $250,000/yr to $343,750/yr. the denver co",250000.0,343750.0,USD,year,denver co
2827,"magic leap 2 platform, including in c/c++ (5+ years) able salary range $123,000$172,000 usd anywhere",123000.0,172000.0,USD,year,anywhere
533,customer networks. assist t1 analysts with triage bachelor's degree and 2 years of prior must have dod 8570 iat ii prior cssp-a certification within 180 days of hire. noscheduled weekly hours: 40shift: rotatingrequisition category: professionaljob . estimated salary: $20 to $28 per st louis mo,20.0,28.0,USD,hour,st louis mo
525,"ensure continuous 24x7 production uptime share in 24x7 on-call responsibilities, in a 24x7 saas environment cybersecurity framework v1.1, nist special special publication 800-53 rev. 5, cloudfront, route 53, cloudwatch, and 800-53 rev. 5, and fips and fips 140-2). experience in in supporting soc2 type ii required. minimum 1-3 years of to include (100% employer-paid dental $80k - $100k based on single coverage), 401(k) with company position is $80k - $100k canada",80000.0,100000.0,USD,year,canada
2582,"full-time, permanent salary: $60,000.00-$80,000.00 per year benefits: (preferred) experience: ros: 1 year (preferred) software (preferred) software development: 2 years (required) work in kingsville, on n9y 0g2 kingsville canada",60000.0,80000.0,CAD,year,kingsville canada
2314,"experience required: 8+ years of home schedule: 8 hour shift professional experience (11+ years in permanent salary: $78,000.00-$100,000.00 per year canada",78000.0,100000.0,CAD,year,canada
1622,contract length: 2 weeks salary: weeks salary: $24.00-$35.00 per hour canada,24.0,35.0,CAD,hour,canada


In [42]:
processed_df.sample(n=10)[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
583,"oh - day 1 onsite... duration: full-time (required) incident support: 1 year (required) break/fix: higher. at least 3 years of relevant messaging/communication/server infrastructure minimum 3 years design and type: full-time pay: $50,000.00 - $70,000.00 per per year benefits: 401(k) dental insurance health vision insurance schedule: 8 hour shift ability commute/relocate: groveport, oh 43125: reliably commute or anywhere",50000.0,70000.0,USD,year
2714,"salaire : 65 000,00$ a : 65 000,00$ a 90 a 90 000,00$ par an 000,00$ a 90 000,00$ par canada",65000.0,90000.0,CAD,year
3027,"or related fields 5+ years of experience offer starting from 3100eur/gross and a yearly during sick leave, 3rd pension pillar contribution, in the office 2-3 times a week, canada",3100.0,3100.0,EUR,month
537,"focus on office/microsoft 365 transition support. this tools to office/microsoft 365 during a multi-year in several office/microsoft 365-related technology implementation teams, products to office/microsoft 365. secondary: deliver limited understanding of office/microsoft 365 core technologies (email, the choice between 3 different health plans; accounts; starting with 22 days of paid in addition to 12 or more paid more paid holidays; 8 weeks of paid disability; up to $4,360 after your first or up to $2,000 for professional development advanced level (tier 2) support with a equivalent at least 5 years of related senior academic recruiter, xer5ff for more information anticipated hiring range: $65,000 to $75,000. this position ending february 2026 beyond that funding anywhere",65000.0,75000.0,USD,year
224,"amazon lambda, amazon s3, amazon ec2, and . estimated salary: $20 to $28 per tampa fl",20.0,28.0,USD,hour
300,"and got a 77% flex score... are decision-making throughout our b2b saas organization. this proven experience of 3+ years in a revenue well over $50 million. python pro: securly in over 20% of the us safety, benefiting over 10+ million children and receive an annual $1,000 stipend for professional we close at 1 pm local time benefit from a 401k plan with employer full-time salary: $ 49,000.00 116,000.00 per year anywhere",49000.0,116000.0,USD,year
4422,"all on x64 our tools: and libvirt 5-20% of your is between $120,000-$145,000 based on canada",120000.0,145000.0,CAD,year
4413,"strategies in 2023. this is within a 1 day trip action with 100m monthly users, and a 100% user-generated revenue to meet 100% of the monthly users, 50,000+ campaigns launched target experience: 4+ years of engineer is $140,500 cad in would be $184,000 usd in as follows: 30 minute screen live coding 30 min interview a recruiter 45 minute interview engineering manager 3 virtual onsite canada",140500.0,140500.0,CAD,year
1310,"the following: 5+ years of react and/or es6+ javasscript, html, - payment 100% in usd.",100.0,100.0,USD,month
4261,"minimum qualifications 8+ years experience stem fields 2+ years domain at fast-growing b2c companies #li-remote below. wa $191,700$213,000 usd or, il, hi $182,700$203,000 usd all other states $166,500$185,000 usd ca, ct, nj $199,800$222,000 usd accommodations anywhere",191700.0,213000.0,USD,year


In [43]:
processed_df[(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")&(processed_df["location"]=="argentina")].sample(n=10)[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
809,salaried position 401k with employer employer match 100% medical/dental/mental health support/vision contributions $20k annual family,20000.0,20000.0,USD,year,argentina
2444,"knoxville national +50 miles away pulsepoint, national +50 miles away work and 0-3 days in minimum of 8 years business position is $79,420 to $150,480. $79,420 to $150,480. locations: california; purchase and 401k contribution (all",79420.0,150480.0,USD,year,argentina
2926,"today, over 2,000 companies and companies and 1 million developers tools. salary: 48: 55k",48000.0,55000.0,USD,year,argentina
639,communication skills. $25 - $31 $25 - $31 an hour,25.0,31.0,USD,hour,argentina
792,"the past 3 years and heroku, aws (s3, cloudfront, & at least 3 years of functions, aws (s3, cloudfront & work a 40-45 hours work at least 5 years of complete it (2 hours maximum) abilities: between $90.000/110.000 usd",90000.0,110000.0,USD,year,argentina
2954,chronicle in 30 days you chronicle in 60 days you build in 90 days and you have 6+ years of of the 0-1 journey have for web 2.0 are comfortable technology (think m2 chip macs) program offers $1000 per year,1000.0,1000.0,USD,year,argentina
2968,"higheredjobs.com. search 2,000+ computerand information role number: 200379076 are you and ml pipelines.\u00a0 contribute to and empower 8116 - midtown new digital 8116 - midtown office - 2220 w. broad richmond, virginia, 23220carmax, the way posted: jun 6, 2023 weekly jun 6, 2023 weekly hours: weekly hours: 40 role number: role number: 200482795 do you aws inferentia-based inf1 and trainium-based and trainium-based trn1 instances. this products, and 1 year there models. qualifications: 7+ we are data structures/algorithms. 7 years of are over 7 billion people area, ca $170,000 - $220,000 $170,000 - $220,000 + competitive of over 15,000 dealers job job description https://careers.oracle.com/jobs/#en/sites/jobsearch/requisitions/preview/202932/?keyword=202932&mode;=location at oracle practical experience. 8 years of language processing. 5 minimum qualifications: language processing. 5 summary posted: practical experience. 5 years of role number: 200478076 the data data structures/algorithms. 3 years of and by 2050, there will will be 2 billion more. data scientist ($225,000 - $275,000 ($225,000 - $275,000 + stock benefits offered: 401k, dental, life,",170000.0,220000.0,USD,year,argentina
2401,developer from 2 years of currently have 15+ frontend openings an initial 15 min call frontend openings 10+ different companies full time 100 remote contract middle level 35k 55k senior level 35k 55k senior 55k 55k senior 55k 75k required level 35k 55k senior 55k 55k senior 55k 75k required senior 55k 75k required exposure calendly/federica erminio 1/30min after an,35000.0,55000.0,USD,year,argentina
1951,"remote - $30k - $50k remote - $30k - $50k $30k - $50k usd go $30k - $50k usd (000000000000385-1) remote - $30k - $50k remote - $30k - $50k $50k usd (000000000000385-1) location remote : minimum 3 years of in a b2b marketing role,",30000.0,50000.0,USD,month,argentina
2839,remote utc -03:00 utc +03:00 -03:00 utc +03:00 more details between utc -03:00 and utc and utc +03:00. i this remote utc -03:00 utc +03:00 between utc -03:00 and utc contractor - 24 months gross annual salary: 30.000 - 90.000 30.000 - 90.000 data scientist at least 5 years of remote utc -03:00 utc +03:00 between utc -03:00 and utc remote utc -03:00 utc +03:00 between utc -03:00 and utc services to 1 client at,30000.0,90000.0,USD,year,argentina


In [44]:
processed_df[processed_df["reduce_description"].str.contains("osde 310")].shape[0]

1

In [45]:
processed_df[processed_df["reduce_description"].str.contains("osde")].shape[0]

3

In [46]:
processed_df[processed_df["reduce_description"].str.contains("osde")][HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
2611,horaria homeworking 1 a 2 1 a 2 veces por pepraga osde 210 lugar de,1.0,2.0,ARS,hour,argentina
3324,usd. osde 310 working from gadgets. beneficios: 3 semanas de vacaciones + 20 dias de,310.0,310.0,USD,month,argentina
1298,"the world's #1 source of delivery of 100% computer-based education education to k-12 students through requirements minimum 5 years of students learn 5x faster and applications minimum 2 years in approach leverages 50+ years of role is $50 usd/hour, which week and 50 weeks per a full-time (40 hours per usd/year assuming 40 hours per equates to $100,000 usd/year assuming up to 15 minutes to job code: lj-4581-br-saopaulo-senioriosdevel.005 anywhere",50.0,50.0,USD,hour,anywhere


In [47]:
processed_df[processed_df["reduce_description"].str.contains("6 months to months to 1 year")].shape[0]

1

In [48]:
round(processed_df.shape[0]/OG_SIZE, 2)

0.12

# last filters

# si dice osde no puede ser ni 210, 310, 410 

In [49]:
osde_df = processed_df[(processed_df["reduce_description"].str.contains("osde"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==310)|(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==210)|(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==410))]

In [51]:
osde_df.shape

(1, 30)

In [52]:
osde_df[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
3324,usd. osde 310 working from gadgets. beneficios: 3 semanas de vacaciones + 20 dias de,310.0,310.0,USD,month,argentina


In [53]:
processed_df = processed_df[~processed_df.job_id.isin(osde_df.job_id)]

In [54]:
processed_df[(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")&(processed_df["location"]=="argentina")].sample(n=10)[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
2433,"denver national +50 miles away pulsepoint, national +50 miles away live with 50 miles of minimum of 12 years of position is $103,440 to $186,192 $103,440 to $186,192 locations: california; purchase and 401k contribution (all",103440.0,186192.0,USD,year,argentina
358,"application with act360 web & it inc. act360 web & applying to act360 web & give the 360 solution: we up to act360 quality work size projects (10 - 1500 is under 10 people, but... (10 - 1500 hours). our are a 100% remote company results. qualifications 4+ years of in jira. 4+ years of home schedule: 8-hour shift monday full-time salary: $405,904.00 - $710,332.00 $405,904.00 - $710,332.00 per month",405904.0,710332.0,USD,month,argentina
2923,sueldo el 1 de cada cada mes 2 a 4 2 a 4 aumentos salariales,1.0,1.0,USD,month,argentina
2926,"today, over 2,000 companies and companies and 1 million developers tools. salary: 48: 55k",48000.0,55000.0,USD,year,argentina
192,up to 500k with multiple minimum of 2 years related,500000.0,500000.0,USD,month,argentina
1031,"team is 100% remote and global management, a16z, accel, dfj, and a 401(k) plan with about you 5+ years of some nice-to-haves 2+ years in or building 0 to 1 0 to 1 products and issues. salary: $140,000 - $200,000 $140,000 - $200,000 base benefits",140000.0,200000.0,USD,year,argentina
1374,"are: since 1998, exadel has currently has 2800+ employees in ideas. requirements: 6+ years experience level minimum b2 english level: company with 30+ offices in offices in 15+ countries. employees given a $1,000 yearly budget",1000.0,1000.0,USD,year,argentina
229,"100% remote job delivery of 100% computer-based education the worlds #1 source of education to k-12 students through requirements minimum 5 years of students learn 5x faster and applications minimum 2 years in approach leverages 50+ years of role is $50 usd/hour, which week and 50 weeks per a full-time (40 hours per usd/year assuming 40 hours per equates to $100,000 usd/year assuming up to 15 minutes to job code: lj-4581-ar-buenosai-seniormobileen.004 mostrar mas",50.0,50.0,USD,hour,argentina
1061,"with over 600 employees around software engineer 100. the position becoming the 1 global provider end framework (angular5+, bonus points afternoons (a 6 hour day) hour day) 35 hour week",100.0,100.0,USD,hour,argentina
1172,with salaries 100% in usd? azure ad b2c custom policies,100.0,100.0,USD,month,argentina


In [55]:
processed_df[processed_df["reduce_description"].str.contains("osde")][HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
2611,horaria homeworking 1 a 2 1 a 2 veces por pepraga osde 210 lugar de,1.0,2.0,ARS,hour,argentina
1298,"the world's #1 source of delivery of 100% computer-based education education to k-12 students through requirements minimum 5 years of students learn 5x faster and applications minimum 2 years in approach leverages 50+ years of role is $50 usd/hour, which week and 50 weeks per a full-time (40 hours per usd/year assuming 40 hours per equates to $100,000 usd/year assuming up to 15 minutes to job code: lj-4581-br-saopaulo-senioriosdevel.005 anywhere",50.0,50.0,USD,hour,anywhere


# Si dice meses u horario no puede ser el mismo 

In [56]:
#  6 months to months to 1 year

exclude_jobs_id = processed_df[(processed_df["reduce_description"].str.contains("6 months to months to 1 year"))&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==6)&(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==12)].job_id

In [57]:
# General
exclude_jobs_list = []

#hs
#horas
#pm
#am
#40/week

for range_min, range_max in [(9, 18), (9, 5), (3, 5), (6, 1), (6, 12), (3, 4)]:
    exclude_jobs_list.append(processed_df[(((processed_df["reduce_description"].str.contains(str(range_min)))&(processed_df["reduce_description"].str.contains(str(range_max))))
                                           )&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==range_min)&(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==range_max)])

In [58]:
# add all excluded jobs OSDE and range hours
exclude_jobs_df = pd.concat(exclude_jobs_list)

discarted_rows_list.append(exclude_jobs_list)

In [59]:
exclude_jobs_df[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
4601,"experience; three (3) to five to five (5) years of of microsoft 365, azure ad; canada",3.0,5.0,CAD,year,canada
3679,you have: 3 - 5 3 - 5 + years vancouver bc canada,3.0,5.0,CAD,year,vancouver bc canada
3886,"vancouver, bc (100% remote) duration duration : 6-12 months!!! job experience requires 6+ years of multiple (over 4) sap implementation anywhere",6.0,12.0,CAD,month,anywhere
677,canada/remote duration: 6-12+ months job working hours: 7.25 hrs daily the puzzle 8+ years' business certification itil v3 certification it anywhere,6.0,12.0,CAD,month,anywhere
794,remote/canada duration: 6-12+ months skills toronto canada,6.0,12.0,CAD,month,toronto canada
2262,duration : 6-12 months location mississauga canada,6.0,12.0,CAD,month,mississauga canada
3661,remote/canada duration: 6-12+ months job job description 8+ years' application experience with 3+ years of tools (python/perl/shell/html/php). 2+ years' with at least 2 years of toronto canada,6.0,12.0,CAD,month,toronto canada


In [60]:
processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

In [61]:
processed_df.shape[0]

6629

# De 13149 pasamos a 13149, perdimos 6

In [62]:
round(processed_df.shape[0]/OG_SIZE, 4)

0.1161

In [63]:
# rango por pais

# Argentina
exclude_list = processed_df[(processed_df["location"].str.contains("Argentina"))&(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>20000)]

In [64]:
exclude_list[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location


# Salarios de 1$ mensual o anual

In [65]:
processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<=1)&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]!="hour")].shape[0]

42

In [66]:
processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<=1)&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]!="hour")].sample(n=10)[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
2634,"every day. 5 + years of which 3 or more swinject, xcode 1+ years of canada",1.00,1.0,CAD,year,canada
1373,minimum of 1 year experience brazil,1.00,1.0,BRL,year,brazil
144,"benefits include 401(k), stock purchase benefits including 401(k), stock purchase position is $16.00 to $26.00. $16.00 to $26.00. the actual ranging from $0.35 per hour hour to $3.00 per hour canada",0.35,3.0,CAD,hour,canada
730,of zero (0) to one to one (1) year of canada,1.00,1.0,CAD,year,canada
3150,a one 1 solution architect seven months 7.25 hours/day this be onsite 3 days of at least 3 years of into dynamics 365 ce and management of d365 ce and microsoft dynamics 365 ce and and the lgbtq2+ community. randstad ontario canada,1.00,1.0,CAD,month,ontario canada
115,part-time salary: from $1.00 per year benefits: year benefits: 401(k) 401(k) matching flexible schedule pittsburgh pa,1.00,1.0,USD,year,pittsburgh pa
1922,and experienced level 3 it systems administrator focus on microsoft 365 and windows server a partir de $1 al mes idioma: colombia,1.00,1.0,COP,month,colombia
2087,remote/canada duration: 1+ year job of two (2) years' experience last two (2) years. skills anywhere,1.00,1.0,CAD,year,anywhere
1877,"interaction is 100% in english, europe. salary: r$1.00 - r$10.00 members. - 2+ years of client requires 2 meetings per work from 9 am to am to 5 pm est. r$1.00 - r$10.00 per month macapa state amapa brazil",1.00,10.0,BRL,month,macapa state amapa brazil
3662,"and got a 81% flex score. sonatype already used by 15 million developers, we skills you'll need 2-5 years of experience of graph databases (neo4j, jgrapht)experience writing compilers stacks and/or languages. $1 - $1 a bogota bogota colombie",1.00,1.0,COP,hour,bogota bogota colombie


# Exclude salary 1 unit currency per != hour

In [67]:
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<=1)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]!="hour")]

In [68]:
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

In [69]:
processed_df.shape[0]

6594

In [70]:
# remove USD salary over 50K per month

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>50000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="month")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]

discarted_rows_list.append(exclude)

In [71]:
exclude.shape

(105, 30)

In [72]:
#Replace instead
exclude.sample(n=10)[HEADERS+["location"]]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,location
280,"up to 80,000k + (b2 80,000k + (b2 german needed) maximum of 2-3 day client for over 15 years; this it consultant 4+ years experience up to 80k+ depending on saskatchewan canada",80000.0,80000.0,CAD,month,saskatchewan canada
2730,"ceo qualifications: solid 1 year 2 years start date: july 10, 2023 salary: $2,500,000 salary: $2,500,000 - $3,000,000 per month ability colombia",2500000.0,3000000.0,USD,month,colombia
3537,"your life) easier: 100% remote work. you learning curve. salary: $3,000,000 per month application bogota bogota colombia",3000000.0,3000000.0,USD,month,bogota bogota colombia
977,"commensurate work experience. 1-2 years experience within (3 days in office/2 days work from this position is $67,000-79,000. the specific rate in the office 5 days per week a full-time position: 37.5 hours per week start time between 7:00am-9:00am. full-time benefits include: retirement plan matching 401(k) plan medical, dental, insurance. we pay 100% of employee health insurance premium and 50% of employee dependents and personal time. 9 paid holidays voluntary anywhere",67000.0,79000.0,USD,month,anywhere
2655,"vision - employer-sponsored 401(k) if you feel type: full-time pay: $3,910,630 per month experience: experience: digital marketing: 1 year (required) customer bogota bogota colombia",3910630.0,3910630.0,USD,month,bogota bogota colombia
759,"salary: $75,000 - $100,000 $75,000 - $100,000 headquartered in required qualifications 2+ years experience uart, spi, i2c, can bus calgary ab canada",75000.0,100000.0,USD,month,calgary ab canada
3937,"de menos de us$500,000, en la que y combinator en 2018, por el programa tener al menos 5 anos de experiencia debes tomarte minimo 5 (si, leiste bien). de nuestros beneficios: 7 semanas de vacaciones options despues de 1 ano. acceso a licencia parental de 12 semanas para la santiago chile",500000.0,500000.0,USD,month,santiago chile
2350,"got a 75.5% flex score. mayo de 2019 por un mas de 300.000 clientes registrados de usd $65 millones de minimo de 3 anos con cloud, rds, s3, ses experiencia anywhere",300000.0,300000.0,USD,month,anywhere
1331,"experiencia minima de 5 anos como full usando tecnologias java 1. 8 o superior: hibernate jpa, ejb 3 y 4, jsf, remuneracion mensual: usd 2 0 0 0, 0 0 0, 0 0 tiempo : quito ecuador",2000000.0,2000000.0,USD,month,quito ecuador
192,up to 500k with multiple minimum of 2 years related,500000.0,500000.0,USD,month,argentina


In [73]:
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

In [74]:
round(processed_df.shape[0]/OG_SIZE, 4)

0.1137

In [75]:
# Exclude hour exesive in usd

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>200)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="hour")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]

discarted_rows_list.append(exclude)

In [76]:
exclude[HEADERS]

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
1156,"a part-time, 100% remote contract (at least 3 years) (e.g. customers. pay: $800.00 - $950.00 $800.00 - $950.00 per hour network architecture: 1 year (preferred",800.0,950.00,USD,hour
2602,"is a 100% remote working part-time pay: $800.00 - $1,000.00 $800.00 - $1,000.00 per hour",800.0,1000.00,USD,hour
2994,"specialty salary: $5,000.00 - $8,000.00 $5,000.00 - $8,000.00 per hour",5000.0,8000.00,USD,hour
3671,"founded in 2012, elastic is bring along: 5+ years of node.js, react) 3rd party cloud experience (required) #li-md1 additional information up to $1500 (or local up to 40 hours each minimum of 16 weeks of request within 24 business hours brazil",1500.0,1500.00,USD,hour
430,at least 2 years of at least 2 hours of range is $600 - $800 $600 - $800 based on shift of 9am-3pm est (us). brazil,600.0,800.00,USD,hour
...,...,...,...,...,...
4649,"monday through friday, 8:00 am - 5:00 travel (up to 10%) by land and/or vision insurance schedule: 10 hour shift 8 requirements minimum of 5 years of system linux, windows databases: db2, mysql software & languages : java, j2ee, sql, html, css, hourly wage , $90,466.10 - 134,342.16 hyundai type: full-time pay: $90,466.10 - $134,342.16 per dental, and vision 401k with employer matching per hour benefits: 401(k) 401(k) matching employee fountain valley, ca 92708: reliably commute or (required) experience: java: 6 years (required) work fountain valley ca",90466.1,134342.16,USD,hour
76,a related field 3 to 5 years service salary range: $80k -- $100k minimum machine learningestimated salary: $20 to $28 per st louis mo,20000.0,28000.00,USD,hour
456,"of more than 20,000, nationally... job title: & recovery division 1188a lance rd norfolk, rd norfolk, va 23510 schedule: flexible schedule tbd up to 20 hours/40 hours/week between between hours of 4:00pm - 8:00am. must (training will be 2 weeks monday - monday - thursday 8am - 4pm) pay 4pm) pay rate: $15.50/hr. job duties: answer staffing visit us: 360 southport circle, suite virginia beach, va 23452 contact us: (text anywhere",20000.0,20000.00,USD,hour
543,"max bill rate: $37.18 - can go is the top 2 required skills that this position? (m-f? 8-5? overtime? on-call?): m-f this person have? 3+ do they need criteria is met $25/hr how many miles contractor be driving? 50 what is the zip? atlanta, ga 30308 how soon will anywhere",3718.0,3718.00,USD,hour


In [77]:
discarted_row_df = create_discarted_df(discarted_rows_list=discarted_rows_list)

/Users/matiasgonzalez/miniforge3/envs/llmsalary/lib/python3.11/site-packages/pandas/core/internals/construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [78]:
discarted_row_df.shape

(7217, 31)

In [79]:
discarted_row_df[pd.isna(discarted_row_df[f"time_lapse_gpt_{VERSION_MODEL}"])][HEADERS].sample(n=10)

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
3095,"got a 77.25% flex score. at least 8 years of home to 300 elektra, asylum, in the 21st century, our salary range $225,000$300,500 salary ranges law. copyright 2023 warner music relevant documents: 2023 benefits at anywhere",225000.0,300500.0,USD,NaN
691,"length of internship: 6+ months with the or related field. 6+ months experience in be vaccinated against covid-19. intel aligns to intel policies for covid-19 including guidance about in us, california: $63,000.00-$166,000.00 salary range dependent springfield il",63000.0,166000.0,USD,NaN
4794,got a 78.5% flex score. of over $98 mn so anywhere,98000000.0,98000000.0,USD,NaN
1667,"was founded in 2015 out of stanford with leading fortune 1000 companies including bank the marketing teams abm50 program launched last vision); 401k with 50% employer match (up abm prospects (fortune 500) collaborate with abm address those needs 1:1 personalized outbound prospecting click rates) requirements 2+ years of experience this position is $70,400-$105,000. the salary offered this position include 90% employer-paid healthcare for match (up to 4%); open pto policy; united states",70400.0,105000.0,USD,NaN
2244,"world, delivering over 25tbps to more than video codecs like h.264/avc. 8+ years of & disability insurance 401(k) maternity & parental candidates. job id: tw7867 #li-remote #remotefriendly our pay per year $134,500 $261,500 usd canada",134500.0,261500.0,USD,NaN
754,"equivalent practical experience. 5 years of experience software products, and 1 year of experience full-time position is $157,000-$235,000 + bonus + atlanta ga",157000.0,235000.0,USD,NaN
1042,"equivalent practical experience. 8 years of experience in more than 200 countries and territories full-time position is $231,000-$339,000 + bonus + waterloo canada",231000.0,339000.0,CAD,NaN
2451,"plc, hmi, level 2 modeling, historians, cyber historians and level 2 systems and interfaces. cyber security, ind 5.0 and interface with digitalization and industry 5.0. influence regional and/or and automation engineering 15 years of progressive experience with industry 4.0 technologies. experience with capability. salary range: $200k -- $250k minimum electrical engineeringestimated salary: $20 to $28 per kennesaw ga",200000.0,250000.0,USD,NaN
2807,is preferred) with 8 years or more of relevant experience. 3 years of specific environment. salary range: $80k -- $100k minimum quantitative analysisestimated salary: $20 to $28 per kansas city ks,80000.0,100000.0,USD,NaN
1651,"generic medicines. our 115,000 colleagues serve people in more than 160 countries. our nutrition reimbursement, the freedom 2 save student debt chain required qualifications 2-5 years experience in this position is $62,700.00 $125,300.00. in specific united states",62700.0,125300.0,USD,NaN


In [80]:
# Too low
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<500)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="month")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<10000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="year")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<3)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="hour")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

# Keep only timelapses with NaN

In [81]:
discarted_row_df = discarted_row_df[pd.isna(discarted_row_df[f"time_lapse_gpt_{VERSION_MODEL}"])]

In [82]:
get_discarted_sample(discarted_df=discarted_row_df, HEADERS=HEADERS)

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
461,"group of over 29,000+ practical problem solvers thinkers in over 30+ countries. our entrepreneurial for role remote: $73,000-$109,000 our full-time, regular are eligible for 401k matching, and vacation covered from day 1 for paid sick anywhere",73000.0,109000.0,USD,NaN
3012,"your resume. salary $90,000 - $120,000 a unity developers/programmers with 4-6 years of experience, unity experience preferably 3d and 2d game vegas nv",90000.0,120000.0,USD,NaN
1989,"organization. qualifications: a 4-year degree in computer as nist, iso 27001, and pci dss. retirement savings plan (401(k)) with company match in concord, ma 01742 job type: full-time type: full-time pay: $80,000.00 - $145.000.00 per united states",80000.0,145000.0,USD,NaN
935,"as assigned requirements: 1. bs or ba experience is required. 2. 3-5+ years of relational databases, ideally db2 5. experience with and documentation processes 6. ability to work team members required. 8. ability to manage requires minimal supervision. 10. ability to express on project tasks. 12. understanding of non-for-profit washington d.c. is 80k-$109k. this is the anywhere",80000.0,109000.0,USD,NaN
1061,"and paid holidays 401k retirement plan minimum experience. requires two (2) to five (5) lifting, pushing/pulling, carrying 50 lbs of supplies requisition id : 4013 compensation details : : $78,000 - $100,000 jefferson city mo",78000.0,100000.0,USD,NaN
4529,"+ at least 5 years' relevant sap management (tm) + s4 hana implementation experience with at least 1 as a lead travel up to 50% based on work current range is $113,000 to $189,000. you and organizational performance #sap23 #ep22 # all boise id",113000.0,189000.0,USD,NaN
2082,user experience qualifications: 5 year's experience with b.s computer science (4-year) or greater background independently salary range: $100k -- $150k minimum software developmentestimated salary: $20 to $28 per needham,100000.0,150000.0,USD,NaN
1353,"to be between $63,410.00 and $68,000.00 per expect up to 80% travel, so be least the preceding 39 months. envision your - up to 3 days per month. to 80% or 4-5 nights at seasonal in education and 5+ years of experience of application, within 7 days of hire. zero employee premiums 401k plan with a behavioral health coaching 15 paid holidays, including new year's day 15 days of accrued anywhere",63410.0,68000.0,USD,NaN
4354,"assistance offered # 154240 - emporia, kansas, when performed at 3rd party location. generate lift up to 50 pounds ability to biology or microbiology 1+ years of quality team! salary range $48,000- $69,300 pay is founded more than 75 years ago with in more than 200 countries. to learn united states",48000.0,69300.0,USD,NaN
3139,32190br requisition id: requisition id: 32190br business unit: bachelor's degree 15 years of required (within 12 months of receive within 12 months of greater than $1m. current pm travel required: 10% assignment category: canada,1000000.0,1000000.0,USD,NaN


In [83]:
discarted_row_df.shape

(6584, 31)

In [84]:
# Test what can be recovered with the new model version
reprocessed_records_list = []

discarted_row_df["location"] = discarted_row_df["location"].fillna(" ")
discarted_row_df= discarted_row_df.dropna(subset="job_id")
total_cost = 0
for _, row in tqdm(discarted_row_df.iterrows(), total=discarted_row_df.shape[0]):
    salary_output = process_request(description=row.description, location=row.location, jobid=create_job_rerun_id(row), ngrams=NGRAMS, overlap_limit=OVERLAP)

    sent_tokens = salary_output["token_count_sent_chat_gpt"]

    cost = float(salary_output["cost"])
    total_cost += cost
    orignal_description_tokens = len(row.description.split(" "))
    reduce_description_len = len(salary_output["reduce_description"].split(" "))
    ratio_sent_original_description = reduce_description_len/orignal_description_tokens

    reprocessed_records_list.append((row.job_id, salary_output["reduce_description"], salary_output["min"], salary_output["max"], salary_output["currency"], salary_output["time_lapse"], salary_output["source"], sent_tokens, orignal_description_tokens, ratio_sent_original_description, cost))

print(f"TOTAL COST OF REPROCESSING: {total_cost}")

  1%|          | 33/6578 [00:00<00:37, 174.51it/s]

[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]


  1%|          | 79/6578 [00:00<00:31, 208.15it/s]

[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]


  2%|▏         | 128/6578 [00:00<00:28, 226.11it/s]

...

[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]
[Cache HIT]


 47%|████▋     | 3105/6578 [16:21<57:42,  1.00it/s]  Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
).
Error in OpenAICallbackHandler.on_retry callback: 'OpenAICallbackHandler' object has no attribute 'on_retry'
 48%|████▊     | 3144/6578 [18:35<1:00:45,  1.06s/it] 

Not a dictionary:  could not convert string to float: 'hour'


 73%|███████▎  | 4830/6578 [52:32<36:36,  1.26s/it]  Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Error in OpenAICallbackHandler.on_retry callback: 'OpenAICallbackHandler' object has no attribute 'on_retry'
 76%|███████▋  | 5016/6578 [1:05:56<28:01,  1.08s/it]    Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Error in OpenAICallbackHandler.on_retry callback: 'OpenAICallbackHandler' object has no attribute 'on_retry'
 90%|█████████ | 5942/6578 [1:34:35<13:57,  1.32s/it]    Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds

TOTAL COST OF REPROCESSING: 7.042588000000016


# This might be saved

In [85]:
len(reprocessed_records_list)

6578

In [86]:
reprocessed_records_list[0]

('eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBEZXZlbG9wZXIgLSBDb25zdWx0YW50IiwiaHRpZG9jaWQiOiI3bXR6b21XQUxYTUFBQUFBQUFBQUFBPT0iLCJ1dWxlIjoidytDQUlRSUNJSlFYSm5aVzUwYVc1aCIsImZjIjoiRW9zQ0Nzd0JRVTFyTW14c1NqZG1PVGQ1VWtOa1ExcHhjMGR3U2pkQ2VUUnFTblZ5VG1STFQyUXRkWFZmTFU5cmIyeFFNSFJ4TUROalQzSk9WbUYyZW5Oc1IxOVhaVXhEY1RsUlNFMVViMDVCUkc0NVdXbFlZelo0UzJRMVRIazVSMEZRYUhab1ZqaDJPV1oxY3pCYVNsVnVZV0Z5ZDIweE4yZE9VWG93T0hST2RUTkJlVWhpTVc5d1pYZzNTakV6Y0RoR1VISnpWWFZwUm5Od1gzbEtVa3BUU1ZKYVpXdExXbGg1YVhFM09XbHdZaTFCYlZOeVJHMXJURVJYTjBKUWRFbG1aazk2ZVZOSFdVeFlaMlF5ZFZjMUVoWXlZbTVMV2xBdGJVNDNSRXd4YzFGUU1qWnhXV0ZCR2lKQlRFVlRPWFZQZFZsM2RVVk1TR2w2TkZwU2MwSjNaMU5HY0Y5QmVERkRRVGhSIiwiZmN2IjoiMyIsImZjX2lkIjoiZmNfMyIsImFwcGx5X2xpbmsiOnsidGl0bGUiOiJBcHBseSBvbiBHbGFzc2Rvb3IiLCJsaW5rIjoiaHR0cHM6Ly93d3cuZ2xhc3Nkb29yLmNvbS9qb2ItbGlzdGluZy9zb2Z0d2FyZS1kZXZlbG9wZXItY29uc3VsdGFudC1kZXZlbG9wbWVudC1nYXRld2F5LUpWX0tPMCwyOV9LRTMwLDQ5Lmh0bT9qbD0xMDA4NDYxNTk5NTc4XHUwMDI2dXRtX2NhbXBhaWduPWdvb2dsZV9qb2JzX2FwcGx5XHUwMDI2dXRtX3NvdXJjZT1nb29nbGVfam

In [87]:
reprocessed_records_df = pd.DataFrame(reprocessed_records_list,
                         columns=[
                                    "job_id",
                                    "reduce_description",
                                    f"salary_min_gpt_{VERSION_MODEL}",
                                    f"salary_max_gpt_{VERSION_MODEL}",
                                    f"currency_gpt_{VERSION_MODEL}",
                                    f"time_lapse_gpt_{VERSION_MODEL}",
                                    f"source_{VERSION_MODEL}",
                                    f"token_count_sent_chat_gpt_{VERSION_MODEL}",
                                    "token_count_original_description",
                                    "token_rate_sent_original_description",
                                    f"gpt_cost_{VERSION_MODEL}"
                                ]
            )

In [88]:
get_discarted_sample(discarted_df=reprocessed_records_df, HEADERS=HEADERS)

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
3964,"of hosts in-depth design of x86/non-x86 platforms and components, including cpus, range for this position is ($180,000 - $225,000 usd) based on united states",180000.0,225000.0,USD,year
2438,"job id: 23617636 the applications development senior programmer and systems partners. qualifications: minimum 5 years of experience in java of experience in working with html5 based javascript/typescript framework and experience ------------------------------------------------------ primary location salary range: $93,760.00 - $140,640.00 ------------------------------------------------------ citi is tampa fl",93760.0,140640.0,USD,
4219,"will be part of a 10 person team that works with been in business for over 15 years, has 150+ employees and to work in dc in 2009 and 2011. networking support technician migration experience, particularly with exchange 2013 and 2016. experience with office of vmware esxi and windows 2012/2016 server the ability to adapt location washington, dc proposed salary $70,000 to $100,000 bowman williams is as one of america's top 200 best staffing firms in america, washington dc",70000.0,100000.0,USD,
2819,"(or related field) and typically 8+ years of related work experience. customers need now. salary range: $79,100.00 - $138,400.00 we value our dental, vision, paid time off, 401(k), life insurance, flexible work schedules, columbia md",79100.0,138400.0,USD,
3251,"their families. with more than 20,000 customers and several major partnerships backend systems and frontend interfaces 4+ years of relevant experience in you need time off plan 100% medical, dental, and vision insurance our backgrounds, and unified by 100% alignment on our mission. research vision insurance coverage fsa plan 401k a values-based culture that invests range for this position is $125,000 to $165,000. we have a anywhere",125000.0,165000.0,USD,
4872,"degree or equivalent practical experience. 15 years of experience in technology channel sales experience at a b2b software company. preferred qualifications: 5 industry. customers in more than 200 countries and territories turn to for this full-time position is $173,000-$254,000 + bonus + equity + sunnyvale ca",173000.0,254000.0,USD,
4763,"or equivalent professional work experience 3 5 years of work experience through media, content and technology. 7,600 wavemakers in 90 countries with this posting may range from $60,000 to $140,000. individual compensation varies costa mesa ca",60000.0,140000.0,USD,
1163,"and work. with 450,000 route fiber miles in more than 60 countries, we deliver quality requisition #: 328961 when applying for salary min : 72540 salary max : jefferson city mo",72540.0,72540.0,USD,
1696,"initial posting date: 07/25/2023 application deadline: 08/15/2023 agency: department coverage. as of july 1, 2023, full oregon health plan benefits of human services salary range: $4,693 - $7,180 position type: employee analyst (operations and policy analyst 2 - bilingual english/spanish candidates encouraged women, individuals with disabilities, veterans, lgbtqia2s+ community members, and others to these skills may receive a 5% differential pay based on the listed areas substitutes for five (5) years of experience. essential attributes may be eligible to receive 5% added to their base salary join a community of over 10,000 employees dedicated to serving and well. enjoy a work-life balance, 11 paid holidays a year, flexible the legislature upon request. in 2021, the oregon legislature passed a healthier oregon program (house bill 3352). the state launched the heathier heathier oregon program on july 1, 2022. the purpose of this available only to adults ages 19-25 and 55 and older. people us department of homeland securitys i-9 form confirming authorization to work at rebecca.a.wilson@odhs.oregon.gov or call at (503) 881-9248. please be sure to nearly any standard. with over 40,000 employe

In [89]:
def filter_rows(processed_df):
    processed_df = processed_df[processed_df[f"currency_gpt_{VERSION_MODEL}"].isin(allowed_currencies)]

    processed_df = processed_df[processed_df[f"time_lapse_gpt_{VERSION_MODEL}"].isin(valid_time_lapses)]

    processed_df = processed_df.astype({f"salary_min_gpt_{VERSION_MODEL}": "float64", f"salary_max_gpt_{VERSION_MODEL}":"float64"})

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>0]

    # 1 unit currency
    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>1]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<1000000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<50000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<20000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<10000000]

    osde_df = processed_df[(processed_df["reduce_description"].str.contains("osde"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==310)|(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==210)|(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==410))]


    processed_df = processed_df[~processed_df.job_id.isin(osde_df.job_id)]

    for range_min, range_max in [(9, 18), (9, 5), (3, 5), (6, 1), (6, 12), (3, 4)]:
        exclude_jobs_list.append(processed_df[(((processed_df["reduce_description"].str.contains(str(range_min)))&(processed_df["reduce_description"].str.contains(str(range_max))))
                                            )&(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==range_min)&(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==range_max)])

    exclude_jobs_df = pd.concat(exclude_jobs_list)

    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<=1)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]!="hour")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>50000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="month")&(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>200)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="hour")&(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    # too low for a salary
    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<2000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="hour")&(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    # Too low
    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<500)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="month")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<10000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="year")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<3)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL}"]=="hour")&((processed_df[f"currency_gpt_{VERSION_MODEL}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    #40 or 45 hours week
    for week_hours in [25, 40, 45]:
        week_hours = processed_df[(processed_df["reduce_description"].str.contains(f"{week_hours} hours"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==week_hours)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==week_hours))]
        processed_df = processed_df[~processed_df.job_id.isin(week_hours.job_id)]

    #years
    for age in range(50):
        years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} years"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==age))]
        processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

    for age in range(5000):
        years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} fastest"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL}"]==age))]
        processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

    return processed_df

In [90]:
filtered_reprocessed_records_df = filter_rows(reprocessed_records_df)
processed_df = filter_rows(processed_df)

In [91]:
filtered_reprocessed_records_df.shape

(933, 11)

In [92]:
get_discarted_sample(discarted_df=filtered_reprocessed_records_df, HEADERS=HEADERS)

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
200,"25 jul 2023 full-time, full-time, contract remote $80,000 / per year and mentoring tier 1 support analysts to position, supporting tier 1 & 2 issues security on tier 2 level. how you tools provide tier 2/3 technical and application analysts through weekly 1:1 meeting track responses, what you bring 3 years of hands-on understanding/experience with microsoft office365, ms azure administration, canada",80000.0,80000.0,USD,year
2845,"titlesap software developer anticipated hiring range$110,000 - $125,000 full time / in accordance with the six (6) phases of the software development sap erp 6.0 (preferably ehp 6+) and hands-on knowledge of multiple of the organization. special skills/knowledge: 4+ years of abap programming experience of sap technologies such as s/4hana, abap restful application programming model abap objects and modern abap (7.40+) is required. recent experience with able to lift up to 15 pounds). must be able to use of a computer and 10-key. work will at times require at times require more than 8 hours per day or an san diego ca",110000.0,125000.0,USD,year
263,"working with fortune 500 clients to solve clean design (web 2.0). professional and/ or readers, bloggers, web 2.0 aficionados, etc. technology permanent role is $50,000-$70,000. the contract range typical range is $45-$60/hour. in addition to of up to 15% of your salary. allowed up to $5,000/year to go anywhere give you a $250 referral fee. go canada",50000.0,70000.0,USD,year
1425,months of hire. at least 3-5 years' experience in information security or system administration. salary range: $80k -- $100k minimum qualification it minimum qualification it securityestimated salary: $20 to $28 per hour based edwardsville il,80000.0,100000.0,USD,year
2257,"as: go hilton travel program: 100 nights of discounted travel with billion guests in our more-than 100-year history. hilton is proud to room rates as low as $40/night hilton shares: our employee stock purchase hilton shares at a 15 percent discount paid parental leave have these minimum qualifications: five (5) years of professional experience within throughout the year, up to 5 bereavement days, flexible spending accounts, or a related field four (4) years of experience with full-cycle leave for birth parents and 4 weeks for non-birth parents, 10 testing, release, and maintenance three (3) years of experience with android and java travel up to 10% it would be useful if degree, or associate's degree plus 6+ years of technology related experience, of full employment, up to 12 weeks of parental leave for of technology related experience seven (7) + years of professional experience a 15 percent discount, a 401(k) savings plan, 20 days of employment and increasing up to 25 days after completing one year parents, 10 paid holidays and 2 floating holidays throughout the year, range for this role is $100,000-$140,000 and is determined based on united states",100000.0,140000.0,USD,year
2916,"world as one of the 12 hot cloud computing companies in in 2011, and currently have 40+ employees in a beautiful new... home is an option after 6 months of employment. this is limited to microsoft, vmware, cisco. 2+ years experience in sys admin/help and company benefits including matching 401(k), health, dental, vision, and quarterly manhattan, new york proposed salary $60,000-$70,000 bowman williams is a national as one of america's top 200 best staffing firms in america, new york ny",60000.0,70000.0,USD,year
755,"of up to $40k offered for this insurance dental insurance $50,000 life disability 2 2 year/60%(employer pays 100% for employee for of the above) 3% retirement match home mansfield oh",40000.0,40000.0,USD,year
4692,"us person as defined by 22 c.f.r. 120.15 is required. us asylee. basic qualifications (required skill/experience): 5 years' experience working on teams or within a business function 3 years of experience 

In [93]:
filtered_reprocessed_records_df[filtered_reprocessed_records_df["salary_min_gpt_0.8.1"]>filtered_reprocessed_records_df["salary_max_gpt_0.8.1"]].shape

(2, 11)

In [94]:
processed_df[processed_df["salary_min_gpt_0.8.1"]>processed_df["salary_max_gpt_0.8.1"]].shape

(4, 30)

In [95]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]<=processed_df[f"salary_max_gpt_{VERSION_MODEL}"]]
filtered_reprocessed_records_df = filtered_reprocessed_records_df[filtered_reprocessed_records_df[f"salary_min_gpt_{VERSION_MODEL}"]<=filtered_reprocessed_records_df[f"salary_max_gpt_{VERSION_MODEL}"]]

In [96]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL}"]>processed_df[f"salary_max_gpt_{VERSION_MODEL}"]].shape

(0, 30)

In [97]:
filtered_reprocessed_records_df[filtered_reprocessed_records_df[f"salary_min_gpt_{VERSION_MODEL}"]>filtered_reprocessed_records_df[f"salary_max_gpt_{VERSION_MODEL}"]].shape

(0, 11)

In [98]:
preprocessed_count = processed_df.shape[0]
rescued_count = filtered_reprocessed_records_df.shape[0]

print(f"Preprocessed: {preprocessed_count}")
print(f"Recued: {rescued_count}")
print(f"% recovered: {rescued_count/preprocessed_count}")
print(f"Salary %: {preprocessed_count/OG_SIZE}")
print(f"Salary % with recovered: {(preprocessed_count+rescued_count)/OG_SIZE}")

Preprocessed: 4211
Recued: 931
% recovered: 0.2210876276418903
Salary %: 0.07376848153598206
Salary % with recovered: 0.09007778011351693


In [99]:
# Join them to the filtered preprocessed?
processed_df = pd.concat([processed_df, filtered_reprocessed_records_df])

In [100]:
get_discarted_sample(discarted_df=processed_df, HEADERS=HEADERS)

,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1
1709,"a dynamic b2b director of throughout the b2b customer journey. leveraging the 6sense platform, including such as 6sense, salesforce, and familiar with 6sense, salesforce, hubspot, someone with 7+ years of with webflow $130,000 - $170,000 salary between $130,000 - $170,000 $130,000 - $170,000 a year $130,000 - $170,000 usd per benefits. requisition #1097 samba tv canada",130000.0,170000.0,USD,year
3839,at least 6 months! competitive rate from $70-$100 cad p/h have: - 2+ years experience canada,70.0,100.0,CAD,hour
1625,"minimum of 2 years of... - ** 40 days of (up to 25,200/year) with great brazil",25200.0,25200.0,BRL,year
1626,"inception in 2010, the company with over 120 employees, the at least 4 hours of technical recruiter 4 - coding have : 3+ years of including html5, css3, and javascript, such as s3, cloudfront, and in developing css3 styles directly your cv 3 - short markup, including html5, css3, and if short-listed 5 - attending for you? $4000 - $6000* $4000 - $6000* per month schedule of 20 hours per to include: 1 - filling click apply 2 - upload hours, requires 100% commitment, this brazil",4000.0,6000.0,BRL,month
3807,python experience - 5 years experience in within the last 3 years - proficiency (preferred) salary range: $100k -- $150k minimum software developmentestimated salary: $20 to $28 per austin tx,100000.0,150000.0,USD,year
3067,"time... salary: $ 100,000$140,000/ yr + benefits. technical field plus 8 or more years minimal direct supervision. 8+ years of experience the following: java 8+ / openjdk java boulder co",100000.0,140000.0,USD,year
3376,"and work. with 450,000 route fiber miles in more than 60 countries, we deliver trust groups (ex m3aawg) across various platforms or equivalent experience 5+ years of experience skills requisition #: 329389 when applying for salary min : 72540 salary max : united states",72540.0,72540.0,USD,year
3646,redux and es6+ javascript develop express and es6+ javascript develop skills salary: 70-80k... depending on vancouver bc canada,70000.0,80000.0,CAD,year
4350,"of experience 1+ years of position is $165,000 - 230,000 $165,000 - 230,000 annually zone position is $157,000 - 219,000 $157,000 - 219,000 annually zone position is $140,000 - 196,000 $140,000 - 196,000 annually this at least 80% of the average of 4 days per at least 4 to 6 4 to 6 times per from the 2nd annual dei anywhere",165000.0,230000.0,USD,year
540,"pulling data from 500+ individual databases across and applied experience 5+ years of experience computers preferred qualifications 3+ years' experience developing, anticipated hiring range: $9,640/month - $9,834/month csu range: $6,249/month - $12,100/month san jose state package typically worth 30-35% of your base specified application period:june 9, 2022 through june through june 26, 2022. this position is university personnel ***** 408-924-2252 csu vaccination policy by phone at 408-924-1501 or by email vaccinated against the covid-19 virus (including all be found at https://calstate.policystat.com/policy/9779821/latest/and questions may be management positions. (e.g. h1-b visas) all san csu executive order 1083 as a condition education amendments of 1972, and certain other san jose province san jose costa rica",9640.0,9834.0,USD,month


In [101]:
processed_df.to_csv(f"exports/purged_with_salary_{EXPORT_NAME}.csv")

In [102]:
processed_df.shape[0]

5142